# What's this
- 主にチーム内での情報共有を目的とした文書ですがせっかくなのでpublicにしています
- このNotebookは本コンペの概要と、提供データの各カラムの説明をまとめていきます


# Riiid! Answer Correctness Prediction
- 学生が問題を問いた時の情報を元に、学生の理解が足りない分野を推定します
- 教育課題へのAIからのアプローチというテーマになります
    > 受賞チームは、2021年2月に開催されるAI教育に関するAAAI-2021ワークショップ（AIを使用したCOVID後の教育を想像する）でモデルを発表するよう招待されます。すべてのコンテスト参加者は、ワークショップに記事を提出できます。
- とのこと
- submitファイルをuploadする形式ではなくkaggleの時系列データapiを利用したコンペ

# Evaluation
- AUCによる

# Files

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

- データが巨大すぎてpandasのread_csvするとメモリ不足になるためdatatableを利用

In [ ]:
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null

In [ ]:
import sklearn.datasets as datasets
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns
import datatable as dt


In [ ]:
pd.options.display.float_format = '{:.20g}'.format

In [ ]:

data_types = {
    'row_id': 'int32',
    'timestamp': 'int64',
    'user_id': 'int64',
    'content_id': 'int16',
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'
}

train_df = dt.fread('../input/riiid-test-answer-prediction/train.csv').to_pandas()

# train_df = train_df.sample(len(train_df)//5,random_state=42)
for column, d_type in data_types.items():
    train_df[column] = train_df[column].astype(d_type) 

# train.csv

In [ ]:
train_df.describe()

In [ ]:
train_df.head()

## train.csv
- row_id: (int64) ID code for the row.

- timestamp: (int64) the time in milliseconds between this user interaction and the first event completion from that user.
  - そのユーザーの直前のアクションからの経過時間(ミリ秒)

- user_id: (int32) ID code for the user.

- content_id: (int16) ID code for the user interaction

- content_type_id: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.
  - イベントがユーザーに提示された質問である場合は0、イベントが講義を見ているユーザーである場合は1。

- task_container_id: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.
  - 質問または講義のバッチのIDコード。 たとえば、ユーザーは、いずれかの説明を表示する前に、3つの質問を連続して表示する場合があります。 これら3つはすべてtask_container_idを共有します。
  
- user_answer: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.
  - 質問に対するユーザーの回答（ある場合）。 講義の場合は、-1をnullとして読み取ります。

- answered_correctly: (int8) if the user responded correctly. Read -1 as null, for lectures.
  - ユーザーが正しく応答した場合。 講義の場合は、-1をnullとして読み取ります。
    - おそらく正答 1 誤答 0

- prior_question_elapsed_time: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.
  - ユーザーが前の質問バンドルの各質問に回答するのにかかった平均時間（ミリ秒単位）。その間の講義は無視されます。 ユーザーの最初の質問バンドルまたは講義ではnullです。 時間は、ユーザーが前のバンドルの各質問を解決するのにかかった平均時間であることに注意してください。

- prior_question_had_explanation: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.
  - ユーザーが前の質問バンドルに回答した後、その間の講義を無視して、説明と正しい回答を見たかどうか。 値は単一の質問バンドル間で共有され、ユーザーの最初の質問バンドルまたは講義ではnullになります。 通常、ユーザーに表示される最初のいくつかの質問は、フィードバックが得られなかったオンボーディング診断テストの一部でした。



In [ ]:
questions_df = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
lectures_df = pd.read_csv('../input/riiid-test-answer-prediction/lectures.csv')
test_df = pd.read_csv('../input/riiid-test-answer-prediction/example_test.csv')

# questions.csv

In [ ]:
questions_df.describe()

In [ ]:
questions_df.head()

## questions.csv
- question_id: foreign key for the train/test content_id column, when the content type is question (0).
  - コンテンツタイプが質問（0）の場合、train / testcontent_id列の外部キー。

- bundle_id: code for which questions are served together.
  - どの質問と同時送られたか

- correct_answer: the answer to the question. Can be compared with the train user_answer column to check if the user was right.
  - 質問への答え。 train user_answer列と比較して、ユーザーが正しいかどうかを確認できます。

- part: the relevant section of the TOEIC test.
  - TOEICテストの関連セクション。

- tags: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.
  - 質問の1つ以上の詳細なタグコード。 タグの意味は提供されませんが、これらのコードは質問をまとめるのに十分です。

# lectures.csv

In [ ]:
lectures_df.describe()

In [ ]:
lectures_df.head()

## lectures.csv
- lecture_id: foreign key for the train/test content_id column, when the content type is lecture (1).
  - コンテンツタイプが講義（1）の場合、train / testcontent_id列の外部キー。
- part: top level category code for the lecture. 
  - 講義のトップレベルのカテゴリコード。
- tag: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.
  - 講義用の1つのタグコード。 タグの意味は提供されませんが、これらのコードは講義をまとめるのに十分です

- type_of: brief description of the core purpose of the lecture
  - 講義の主な目的の簡単な説明

# example_test_rows.csv

In [ ]:
test_df.describe()

In [ ]:
test_df.head()

# example_test_rows.csv
- Three sample groups of the test set data as it will be delivered by the time-series API. The format is largely the same as train.csv. There are two different columns that mirror what information the AI tutor actually has available at any given time, but with the user interactions grouped together for the sake of API performance rather than strictly showing information for a single user at a time. Some users will appear in the hidden test set that have NOT been presented in the train set, emulating the challenge of quickly adapting to modeling new arrivals to a website.
  - 時系列APIによって配信されるテストセットデータの3つのサンプルグループ。 形式はtrain.csvとほぼ同じです。 AIチューターがいつでも実際に利用できる情報を反映する、2つの異なる列がありますが、一度に1人のユーザーの情報を厳密に表示するのではなく、APIパフォーマンスのためにユーザーインタラクションをグループ化します。 一部のユーザーは、trainセットに表示されていない非表示のテストセットに表示され、Webサイトへの新着のモデリングにすばやく適応するという課題をエミュレートします。

- prior_group_responses (string) provides all of the user_answer entries for previous group in a string representation of a list in the first row of the group. All other rows in each group are null. If you are using Python, you will likely want to call eval on the non-null rows. Some rows may be null, or empty lists.
  - グループの最初の行のリストの文字列表現で、前のグループのすべてのuser_answerエントリを提供します。 各グループの他のすべての行はnullです。 Pythonを使用している場合は、null以外の行でevalを呼び出すことをお勧めします。 一部の行はnullまたは空のリストである可能性があります。

- prior_group_answers_correct (string) provides all the answered_correctly field for previous group, with the same format and caveats as prior_group_responses. Some rows may be null, or empty lists.
  - 以前のグループのすべてのanswered_correctlyフィールドを、prior_group_responsesと同じ形式と警告で提供します。 一部の行はnullまたは空のリストである可能性があります。

#  